In [4]:
import numpy as np
A_prim = np.array([1,1,1])
B_prim = np.array([1,1,1])
A_prim @ B_prim.T

3

In [11]:
import numpy as np

# Read the file and split into lines
with open('sample.txt', 'r') as file:
    lines = file.read().splitlines()

# Convert each line into a list of characters
char_arrays = [list(line) for line in lines]

# Convert the list of lists into a NumPy array
result = np.array(char_arrays)

starting_position = np.where(result == "^")
starting_position = starting_position[0][0], starting_position[1][0]
starting_position

def cell_right(starting_position):
    return starting_position[0], starting_position[1]+1

def cell_left(starting_position):
    return starting_position[0], starting_position[1]-1

def cell_above(starting_position):
    return starting_position[0]-1, starting_position[1]

def cell_below(starting_position):
    return starting_position[0]+1, starting_position[1]


def rotate(character):
    if character == "^":
        return ">"
    elif character == ">":
        return "v"
    elif character == "v":
        return "<"
    elif character == "<":
        return "^"
    else:
        print("Error! Not valid character")


def move(input_array, position):
    # Right
    if input_array[position] == ">":
        if input_array[cell_right(position)] == "#":
            input_array[position] = rotate(input_array[position])
            return input_array, position
        else:
            input_array[cell_right(position)] = ">"
            input_array[position] = "X"
            new_position = cell_right(position)
            return input_array, new_position

    # Left
    elif input_array[position] ==  "<":
        if input_array[cell_left(position)] == "#":
            input_array[position] = rotate(input_array[position])
            return input_array, position
        else: 
            input_array[cell_left(position)] = "<"
            input_array[position] = "X"
            new_position = cell_left(position)
            return input_array, new_position

    # Up
    elif input_array[position] ==  "^":
        if input_array[cell_above(position)] == "#":
            input_array[position] = rotate(input_array[position])
            return input_array, position
        else:
            input_array[cell_above(position)] = "^"
            input_array[position] = "X"
            new_position = cell_above(position)
            return input_array, new_position
    # Down
    elif input_array[position] ==  "v":
        if input_array[cell_below(position)] == "#":
            input_array[position] = rotate(input_array[position])
            return input_array, position
        else:
            input_array[cell_below(position)] = "v"
            input_array[position] = "X"
            new_position = cell_below(position)
            return input_array, new_position

result, starting_position

(array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
        ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U1'),
 (6, 4))

In [12]:
max_below = result.shape[0]-1
max_right = result.shape[1]-1

next_move = move(result.copy(), starting_position)
while True:
    
    next_move = move(next_move[0], next_move[1])
    if next_move[1][0] == max_below:
        print("moved of screen below")
        break
    elif next_move[1][0] == 0 :
        print("moved of screen above")
        break
    elif next_move[1][1] == 0:
        print("moved of screen left")
        break
    elif next_move[1][1] == max_right:
        print("moved of screen right")
        break
    
(next_move[0] == "X").sum() + 1

moved of screen below


41

# Part 2

In [22]:
def look_ahead_next_obstacle(position, input_array, direction):
    # Down:
    if direction == "v":
        obstacle_in_direction = np.isin("#", input_array[position[0]:, position[1]])
        if obstacle_in_direction:
            next_obstacle = np.where(input_array[position[0]:, position[1]] == "#")[0][0]
            print("down", input_array[position[0]:next_obstacle, position[1]])
            return np.all(input_array[position[0]:next_obstacle, position[1]] == "X")
        return False

    #up
    elif direction == "^":
        obstacle_in_direction = np.isin("#", input_array[:position[0], position[1]])
        if obstacle_in_direction:
            next_obstacle = np.where(input_array[:position[0], position[1]] == "#")[0][0] + 1
            print("up", input_array[next_obstacle:position[0], position[1]])
            return  np.all(input_array[next_obstacle:position[0], position[1]] == "X")
        return False

    # right
    elif direction == ">":
        obstacle_in_direction = np.isin("#", input_array[position[0], position[1]:])
        if obstacle_in_direction:
            next_obstacle = np.where(input_array[position[0], position[1]:] == "#")[0][0]
            print("right", input_array[position[0], position[1]:next_obstacle])
            return np.all(input_array[position[0], position[1]:next_obstacle] == "X")
        return False
    
    # left
    elif direction == "<":
        obstacle_in_direction = np.isin("#", input_array[position[0], :position[1]])
        if obstacle_in_direction:
            next_obstacle = np.where(input_array[position[0], :position[1]] == "#")[0][0]
            print("left", input_array[position[0], next_obstacle:position[1]])
            return np.all(input_array[position[0], next_obstacle:position[1]] == "X")
        return False

In [23]:
def move(input_array, position, Loop = False):
    # Right
    if input_array[position] == ">":
        if input_array[cell_right(position)] == "#":
            input_array[position] = rotate(input_array[position])
            if input_array[cell_below(position)] == "X":
                return input_array, position, look_ahead_next_obstacle(position, input_array, "v")
            return input_array, position, False
        else:
            input_array[cell_right(position)] = ">"
            input_array[position] = "X"
            new_position = cell_right(position)
            return input_array, new_position, False

    # Left
    elif input_array[position] ==  "<":
        if input_array[cell_left(position)] == "#":
            input_array[position] = rotate(input_array[position])
            if input_array[cell_above(position)] == "X":
                return input_array, position, look_ahead_next_obstacle(position, input_array, "^")
            return input_array, position, False
        else: 
            input_array[cell_left(position)] = "<"
            input_array[position] = "X"
            new_position = cell_left(position)
            return input_array, new_position, False

    # Up
    elif input_array[position] ==  "^":
        if input_array[cell_above(position)] == "#":
            input_array[position] = rotate(input_array[position])
            if input_array[cell_right(position)] == "X":
                return input_array, position, look_ahead_next_obstacle(position, input_array, ">")
            return input_array, position, False
        else:
            input_array[cell_above(position)] = "^"
            input_array[position] = "X"
            new_position = cell_above(position)
            return input_array, new_position, False
    # Down
    elif input_array[position] ==  "v":
        if input_array[cell_below(position)] == "#":
            input_array[position] = rotate(input_array[position])
            if input_array[cell_left(position)] == "X":
                return input_array, position, look_ahead_next_obstacle(position, input_array, "<")
            return input_array, position, False
        else:
            input_array[cell_below(position)] = "v"
            input_array[position] = "X"
            new_position = cell_below(position)
            return input_array, new_position, False

def check_location(result, starting_position):
    max_below = result.shape[0]-1
    max_right = result.shape[1]-1

    next_move = move(result, starting_position, False)
    while True:
        
        next_move = move(next_move[0], next_move[1], next_move[2])
        if next_move[1][0] == max_below:
            return 0
        elif next_move[1][0] == 0 :
            return 0
        elif next_move[1][1] == 0:
            return 0
        elif next_move[1][1] == max_right:
            return 0
        elif next_move[2]:
            return 1

In [24]:
creates_loop = 0
for row in range(result.shape[0]):
    for col in range(result.shape[1]):
        if result[row, col] == "#" or (row, col) == starting_position:
            continue
        else:
            new_input = result.copy()
            new_input[row, col] = "#"
            loop = check_location(new_input, starting_position)
            creates_loop += loop
print(creates_loop)

up ['X' 'X' 'X' 'X' 'X']
left ['#' 'X' 'X' 'X' 'X']
up ['X' 'X']
down []
right ['>' 'X' 'X']
down ['v']
left ['#' '#' 'X' 'X' 'X' 'X']
up ['X' 'X' 'X' 'X']
right ['>']
down ['v' 'X' 'X' 'X' 'X']
left ['#' 'X' 'X' 'X' 'X' 'X' 'X']
up ['X' 'X']
left ['#' 'X' 'X' 'X' 'X' 'X' 'X']
up ['X']
6


In [13]:
test_map = result.copy()
test_map[6, 3] = "#"
test_map

check_location(test_map, starting_position)

In [25]:
a = np.array([1,2,3,4,5,6,7,8])
a

array([1, 2, 3, 4, 5, 6, 7, 8])

In [32]:
a[6:1:-1]

array([7, 6, 5, 4, 3])